# Hello, Colab!

Let's quickly explore our Google Colab environment.

### Colab Notebook

This is just a Jupyter Notebook that is opened in [Google Colab](https://colab.research.google.com/).
It mixes Markdown and executable Python in the same document and then allows you to run on Google Cloud for free!

#### To open this notebook in Colab

Either:

1. Link GitHub account and browse to this file
2. Install the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension
3. Change your URL to replace `github.com/` with `githubtocolab.com`
4. Download the file and then upload it to Colab

## Platform

In [ ]:
import platform
print(platform.platform())

import sys
print("Python version:", sys.version)

## CPU

In [ ]:
import os

cpu_cores = os.cpu_count()

print("Number of cores:", cpu_cores)

## GPU

We have to enable the GPU first, as described [here](https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=oM_8ELnJq_wd)


- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down


In [ ]:
%pip install GPUtil

In [ ]:
import GPUtil

gpus = GPUtil.getGPUs()
for gpu in gpus:
    print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


## Tensorflow on the GPU

Let's confirm that we can import Tensorflow and it can find the GPU.

If this doesn't work, see the previous step and enable the GPU in **Edit -> Notebook Settings**

In [ ]:
import tensorflow as tf

print("Running Tensorflow version", tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

### Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a random image and manually places the resulting ops on either the CPU or the GPU to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))